In [1]:
from pyspark.sql import DataFrame, SparkSession

spark: SparkSession = (
    SparkSession.builder.master("local[3]")  # type: ignore
    .appName("Aggregate Example")
    .getOrCreate()
)

spark.version

25/04/08 13:00:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.5.5'

In [2]:
invoice_df: DataFrame = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/invoices.csv")
)

invoice_df.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     NULL|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



# Simple Aggregations

summarize whole dataframe to get 1 single row in the result.

> these aggregates are functions, so we can use them in column object expression or SQL like string expression.


In [3]:
from pyspark.sql import functions as F

invoice_df.select(
    F.count("*").alias("Total Count"),
    F.sum("Quantity").alias("Total Quantity"),
    F.avg("UnitPrice").alias("Average Unit Price"),
    F.count_distinct("InvoiceNo").alias("Distinct Invoice Count"),
).show()

[Stage 3:===================>                                       (1 + 2) / 3]

+-----------+--------------+------------------+----------------------+
|Total Count|Total Quantity|Average Unit Price|Distinct Invoice Count|
+-----------+--------------+------------------+----------------------+
|     541909|       5176450| 4.611113626088481|                 25900|
+-----------+--------------+------------------+----------------------+



In [4]:
invoice_df.selectExpr(
    "COUNT(1) AS `Total Count`",  # count even null values, COUNT(*)
    "COUNT(StockCode) AS `Total Stock Count`",  # count non-null values
    "SUM(Quantity) AS `Total Quantity`",
    "AVG(UnitPrice) AS `Average Unit Price`",
).show()

[Stage 9:===================>                                       (1 + 2) / 3]

+-----------+-----------------+--------------+------------------+
|Total Count|Total Stock Count|Total Quantity|Average Unit Price|
+-----------+-----------------+--------------+------------------+
|     541909|           541908|       5176450| 4.611113626086849|
+-----------+-----------------+--------------+------------------+



## Exercise 1

- based on this [_dataset_](code/01-Aggregate/data/invoices.csv)

| Country        | InvoiceNo | TotalQuantity | InvoiceValue |
|----------------|-----------|---------------|--------------|
| United Kingdom | 536446    | 329           | 440.89       |
| United Kingdom | 536508    | 216           | 155.52       |
| United Kingdom | 537018    | -3            | 0.0          |
| United Kingdom | 537401    | -24           | 0.0          |
| United Kingdom | 537811    | 74            | 268.86       |

- group by `Country` and `InvoiceNo`
- `InvoiceValue` is sum of [`Quantity` * `UnitPrice`]

In [5]:
# using spark.sql
invoice_df.createOrReplaceTempView("invoices")

exercise1 = spark.sql(
    """
    SELECT
        Country,
        InvoiceNo,
        SUM(Quantity) AS TotalQuantity,
        ROUND(SUM(Quantity * UnitPrice), 2) AS InvoiceValue
    FROM
        invoices
    GROUP BY
        Country,
        InvoiceNo
    """
)

exercise1.show(5)

[Stage 12:======================================>                   (2 + 1) / 3]

+--------------+---------+-------------+------------+
|       Country|InvoiceNo|TotalQuantity|InvoiceValue|
+--------------+---------+-------------+------------+
|United Kingdom|   536446|          329|      440.89|
|United Kingdom|   536508|          216|      155.52|
|United Kingdom|   537018|           -3|         0.0|
|United Kingdom|   537401|          -24|         0.0|
|United Kingdom|   537811|           74|      268.86|
+--------------+---------+-------------+------------+
only showing top 5 rows



In [6]:
# OR using pyspark.sql.functions

# agg(*exprs: Column) -> DataFrame
# Compute aggregates and returns the result as a `DataFrame`.

exercise1_alt = invoice_df.groupBy("Country", "InvoiceNo").agg(
    F.sum("Quantity").alias("TotalQuantity"),
    F.expr("ROUND(SUM(Quantity * UnitPrice), 2)").alias("InvoiceValue"),
    # OR
    F.round(F.sum(F.expr("Quantity * UnitPrice")), 2).alias("InvoiceValueExpr"),
)

exercise1_alt.show(5)

[Stage 15:===================>                                      (1 + 2) / 3]

+--------------+---------+-------------+------------+----------------+
|       Country|InvoiceNo|TotalQuantity|InvoiceValue|InvoiceValueExpr|
+--------------+---------+-------------+------------+----------------+
|United Kingdom|   536446|          329|      440.89|          440.89|
|United Kingdom|   536508|          216|      155.52|          155.52|
|United Kingdom|   537018|           -3|         0.0|             0.0|
|United Kingdom|   537401|          -24|         0.0|             0.0|
|United Kingdom|   537811|           74|      268.86|          268.86|
+--------------+---------+-------------+------------+----------------+
only showing top 5 rows



# Exercise 2

| Country         | WeekNumber | NumInvoices | TotalQuantity | InvoiceValue |
|-----------------|------------|-------------|---------------|--------------|
| Australia       | 50         | 2           | 133           | 387.95       |
| Australia       | 48         | 1           | 107           | 358.25       |
| Australia       | 49         | 1           | 214           | 258.9        |
| Austria         | 50         | 2           | 3             | 257.04       |
| Bahrain         | 51         | 1           | 54            | 205.74       |
| Belgium         | 48         | 1           | 528           | 346.1        |
| Belgium         | 50         | 2           | 285           | 625.16       |
| Belgium         | 51         | 2           | 942           | 838.65       |

- group by `Country`, `WeekNumber` _(get week number from InvoiceDate)_
- show records of year `2010` only.
- NumInvoices are distinct invoices in that group by
- `InvoiceValue` is sum of [`Quantity` * `UnitPrice`]

In [7]:
invoice_df.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     NULL|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

In [8]:
from pyspark.sql.functions import col, to_timestamp

invoice_df.show()

# 1. Convert the InvoiceDate string column to Timestamp type
#    The format "dd-MM-yyyy HH.mm" must exactly match your string data.
invoice_df_typed = invoice_df.withColumn(
    "InvoiceDate",
    to_timestamp(col("InvoiceDate"), "dd-MM-yyyy H.mm")
)

invoice_df_typed.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     NULL|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

In [9]:
# 2. Register the correctly typed DataFrame as a temporary view
invoice_df_typed.createOrReplaceTempView("exercise_2")

# 3. Execute the SQL query
#    It will now work because InvoiceDate is a Timestamp in the view.
exercise2DF = spark.sql(
    """
    SELECT
        Country,
        weekofyear(InvoiceDate) AS WeekNumber,  -- weekofyear is not in PostgreSQL
        COUNT(DISTINCT(InvoiceNo)) AS NumInvoices,
        SUM(Quantity) AS TotalQuantity,
        ROUND(SUM(Quantity * UnitPrice), 2) AS InvoiceValue
    FROM
        exercise_2
    WHERE
        year(InvoiceDate) == 2010
    GROUP BY
        Country,
        WeekNumber
    ORDER BY
        Country, WeekNumber
    """
)

exercise2DF.show()

[Stage 21:======================================>                   (2 + 1) / 3]

+---------------+----------+-----------+-------------+------------+
|        Country|WeekNumber|NumInvoices|TotalQuantity|InvoiceValue|
+---------------+----------+-----------+-------------+------------+
|      Australia|        48|          1|          107|      358.25|
|      Australia|        49|          1|          214|       258.9|
|      Australia|        50|          2|          133|      387.95|
|        Austria|        50|          2|            3|      257.04|
|        Bahrain|        51|          1|           54|      205.74|
|        Belgium|        48|          1|          528|       346.1|
|        Belgium|        50|          2|          285|      625.16|
|        Belgium|        51|          2|          942|      838.65|
|Channel Islands|        49|          1|           80|      363.53|
|         Cyprus|        50|          1|          917|     1590.82|
|        Denmark|        49|          1|          454|      1281.5|
|           EIRE|        48|          7|        

In [10]:
# OR

from pyspark.sql import functions as f


NumInvoices = f.count_distinct("InvoiceNo").alias("NumInvoices")
TotalQuantity = f.sum("Quantity").alias("TotalQuantity")
InvoiceValue = f.expr("ROUND(SUM(Quantity * UnitPrice), 2) AS InvoiceValue")

summaryDF = (
    invoice_df.withColumn("InvoiceDate", f.to_date(f.col("InvoiceDate"), "dd-MM-yyyy H.mm"))
    .where("year(InvoiceDate) == 2010")
    .withColumn("WeekNumber", f.weekofyear(f.col("InvoiceDate")))
    .groupBy("Country", "WeekNumber")
    .agg(NumInvoices, TotalQuantity, InvoiceValue)
)

summaryDF.sort("Country", "WeekNumber").show()

# summaryDF.coalesce(1).write.format("parquet").mode("overwrite").save("output")
summaryDF.coalesce(1).write.format("csv").mode("overwrite").save("output")

+---------------+----------+-----------+-------------+------------+
|        Country|WeekNumber|NumInvoices|TotalQuantity|InvoiceValue|
+---------------+----------+-----------+-------------+------------+
|      Australia|        48|          1|          107|      358.25|
|      Australia|        49|          1|          214|       258.9|
|      Australia|        50|          2|          133|      387.95|
|        Austria|        50|          2|            3|      257.04|
|        Bahrain|        51|          1|           54|      205.74|
|        Belgium|        48|          1|          528|       346.1|
|        Belgium|        50|          2|          285|      625.16|
|        Belgium|        51|          2|          942|      838.65|
|Channel Islands|        49|          1|           80|      363.53|
|         Cyprus|        50|          1|          917|     1590.82|
|        Denmark|        49|          1|          454|      1281.5|
|           EIRE|        48|          7|        